In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_columns

20

### Creating the working set (FAO Data)

In [2]:
#crop_harvest = pd.read_csv("./raw_data/fao_data/fao_harvest.csv")
#crop_prod = pd.read_csv("./raw_data/fao_data/fao_prod.csv")
crop_yield = pd.read_csv("./raw_data/fao_data/fao_yield.csv")
change_temp = pd.read_csv("./raw_data/fao_data/fao_temp.csv")

In [3]:
# crop_harvest = crop_harvest.loc[(crop_harvest["Flag"] == "A") | (crop_harvest["Flag"] == "E")]
# crop_harvest = crop_harvest.reset_index(drop=True)
# crop_harvest = crop_harvest.drop_duplicates()

# crop_prod = crop_prod.loc[(crop_prod["Flag"] == "A") | (crop_prod["Flag"] == "E")]
# crop_prod = crop_prod.reset_index(drop=True)
# crop_prod = crop_prod.drop_duplicates()

#crop_yield = crop_yield.loc[(crop_yield["Flag"] == "A") | (crop_yield["Flag"] == "E")]
crop_yield = crop_yield.reset_index(drop=True)
crop_yield = crop_yield.drop_duplicates()
crop_yield["Value"] = crop_yield["Value"].apply(lambda x: x / 10_000)

#change_temp = change_temp.loc[(change_temp["Flag"] == "A") | (change_temp["Flag"] == "E")]
change_temp = change_temp.reset_index(drop=True)
change_temp = change_temp.drop_duplicates()
change_temp = change_temp[change_temp["Element"] == "Temperature change"]

In [4]:
# crop_harvest = crop_harvest.rename(columns={"Value":"Harvest (ha)"})
# crop_prod = crop_prod.rename(columns={"Value":"Production (tonnes)"})
crop_yield = crop_yield.rename(columns={"Value":"Yield (tonnes/ha)"})
change_temp = change_temp.rename(columns={"Value":"Change (C)"})

In [5]:
# Combines both dataframes and creates the final FAO dataframe
## Keep 5 columns, year item Harvest (ha) Production (tonnes) Yiel (tonnes/ha)
final_fao = crop_yield[["Year", "Item", "Yield (tonnes/ha)"]].merge(change_temp[["Year", "Change (C)"]], on=["Year"], how="inner")
#final_fao = final_fao.merge(crop_prod[["Year","Item", "Production (tonnes)"]], on=["Year", "Item"], how="inner")
#final_fao = final_fao.merge(change_temp[["Year", "Change (C)"]], on="Year", how="inner")
final_fao

,Year,Item,Yield (tonnes/ha),Change (C)
0,1985,Apples,18.8887,-0.804
1,1985,Apricots,2.0624,-0.804
2,1985,Asparagus,2.6121,-0.804
3,1985,Barley,4.9648,-0.804
4,1985,"Beans, dry",3.5945,-0.804
...,...,...,...,...
2786,1984,Sugar beet,43.1721,-0.115
2787,1984,Tomatoes,24.7879,-0.115
2788,1984,Unmanufactured tobacco,1.9279,-0.115
2789,1984,Vetches,1.1027,-0.115


In [6]:
# Overlook of data
final_fao[(final_fao["Year"] == 2021)].sample(10)# & final_data["Item"] == "Wheat"]#.iloc[0]

,Year,Item,Yield (tonnes/ha),Change (C)
1854,2021,Raspberries,6.8824,1.304
1846,2021,"Other vegetables, fresh n.e.c.",10.6545,1.304
1863,2021,Triticale,5.8136,1.304
1845,2021,Other berries and fruits of the genus vacciniu...,3.1160,1.304
1861,2021,Sunflower seed,2.6031,1.304
1842,2021,Oats,4.3232,1.304
1822,2021,Apples,29.5653,1.304
1864,2021,Wheat,7.3015,1.304
1823,2021,Asparagus,5.3532,1.304
1858,2021,Spinach,20.2685,1.304


### Creating the working Subset (DWD Data)

In [7]:
precip = pd.read_csv("./raw_data/precipitation/observed_average_annual_precipitation.csv")
air_temp_mean = pd.read_csv("./raw_data/air_temperature/average_air_temperature.txt", sep=";", index_col=False)
frost_days = pd.read_csv("./raw_data/frost_days/regional_averages_tnas_year.txt", sep=";", skiprows=1)
hot_days = pd.read_csv("./raw_data/hot_days/regional_averages_txbs_year.txt", sep=";", skiprows=1)
ice_days = pd.read_csv("./raw_data/ice_days/regional_averages_txcs_year.txt", sep=";", skiprows=1)
summer_days = pd.read_csv("./raw_data/summer_days/regional_averages_txas_year.txt", sep=";", skiprows=1)#, index_col=False) 
sunshine_duration = pd.read_csv("./raw_data/sunshine_duration/regional_averages_sd_year.txt", sep=";", skiprows=1) # hours
tropical_nights = pd.read_csv("./raw_data/tropical_nights/regional_averages_tnes_year.txt", sep=";",skiprows=1)

In [8]:
air_temp_mean.shape

(142, 19)

In [9]:
# Changing hours to days
sunshine_duration["Deutschland"] = sunshine_duration["Deutschland"].apply(lambda x: (x / 24) / 365)

In [10]:
# creating sub dataframes for ease of use
de_pre = precip[["Category", "Annual Mean"]]
de_pre = de_pre.rename(columns={"Annual Mean":"Annual Mean Precipitation"})
de_temp = air_temp_mean[["Jahr", "Deutschland"]]
de_temp = de_temp.rename(columns={"Deutschland":"Air Temp Mean"})
de_frost = frost_days[["Jahr", "Deutschland"]]
de_frost = de_frost.rename(columns={"Deutschland":"Frost Days"})
de_hot = hot_days[["Jahr", "Deutschland"]]
de_hot = de_hot.rename(columns={"Deutschland":"Hot Days"})
de_ice = ice_days[["Jahr", "Deutschland"]]
de_ice = de_ice.rename(columns={"Deutschland":"Ice Days"})
de_sum = summer_days[["Jahr", "Deutschland"]]
de_sum = de_sum.rename(columns={"Deutschland":"Summer Days"})
de_sun = sunshine_duration[["Jahr", "Deutschland"]]
de_sun = de_sun.rename(columns={"Deutschland":"Sunshine Duration"})
de_trop = tropical_nights[["Jahr", "Deutschland"]]
de_trop = de_trop.rename(columns={"Deutschland":"Tropical Nights"})

In [11]:
## Merging all of the sub dataframes
final_dwd = de_pre.merge(de_temp, how="inner", left_on="Category", right_on="Jahr").merge(de_frost, how="inner").\
                    merge(de_hot, how="inner").merge(de_ice, how="inner").merge(de_sum, how="inner").merge(de_sun, how="inner").merge(de_trop, how="inner")

In [12]:
# master_dwd = master_dwd.set_index("Jahr", drop=True)
final_dwd = final_dwd.drop(columns=["Category"])
final_dwd = final_dwd.rename(columns={"Jahr":"Year"})

In [13]:
final_dwd

,Annual Mean Precipitation,Year,Air Temp Mean,Frost Days,Hot Days,Ice Days,Summer Days,Sunshine Duration,Tropical Nights
0,714.44,1951,8.72,85.57,3.02,9.81,27.53,0.195400,0.01
1,757.37,1952,7.94,109.43,7.91,25.70,30.70,0.179989,0.75
2,575.68,1953,8.94,94.54,5.08,23.40,32.96,0.203642,0.01
3,806.06,1954,7.70,96.70,2.53,33.25,17.18,0.178128,0.01
4,719.08,1955,7.54,117.22,0.93,31.93,21.40,0.186495,0.00
...,...,...,...,...,...,...,...,...,...
66,814.74,2017,9.58,76.17,6.80,16.63,39.01,0.182203,0.04
67,571.21,2018,10.45,77.35,20.37,14.64,74.72,0.230068,1.35
68,676.23,2019,10.28,69.74,16.97,7.69,52.29,0.209384,0.82
69,661.60,2020,10.43,62.51,11.39,3.71,45.37,0.216438,0.74


### Random Data

In [14]:
co2 = np.loadtxt(r"./raw_data/co2_mm_mlo.txt")
co2 = pd.DataFrame(data=co2, columns=["Year","Month","decimal date", "monthly average", "de-seasonalized", " number of days", "st.dev of days", "unc. of monthly mean"])
co2["Year"] = co2["Year"].astype(int)

In [15]:
co2.dtypes

Year                      int32
Month                   float64
decimal date            float64
monthly average         float64
de-seasonalized         float64
 number of days         float64
st.dev of days          float64
unc. of monthly mean    float64
dtype: object

In [16]:
co2 = co2[["Year", "monthly average"]]
co2 = co2.rename(columns={"monthly average":"Mean CO2 (ppm)"})
co2 = co2.groupby("Year", as_index=False).mean()

In [17]:
co2

,Year,Mean CO2 (ppm)
0,1958,315.232000
1,1959,315.980833
2,1960,316.910000
3,1961,317.644167
4,1962,318.454167
...,...,...
61,2019,411.654167
62,2020,414.238333
63,2021,416.450833
64,2022,418.564167


### ESWD & ESSL

In [18]:
essl = pd.read_csv("./raw_data/essl_data.csv")

C:\Users\johna\AppData\Local\Temp\ipykernel_84\2851792757.py:1: DtypeWarning: Columns (6,16,18,22,31,38,50,52,60,77,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  essl = pd.read_csv("./raw_data/essl_data.csv")


In [19]:
essl.shape

(44248, 97)

In [20]:
essl["TIME_EVENT"] = essl["TIME_EVENT"].astype("datetime64[Y]")
essl["TIME_EVENT"] = essl["TIME_EVENT"].dt.year
essl["TIME_EVENT"] = essl["TIME_EVENT"].astype(int)

In [21]:
essl_modified = essl[["TIME_EVENT",
       "COUNTRY", 'TYPE_EVENT',
       'NO_OBJECTS', 'MAX_HAIL_DIAMETER', 'MAX_HAILSTONE_WEIGHT',
       'AVERAGE_HAIL_DIAMETER', 'THICKNESS_HAIL_LAYER', 'HAILSTONE', 'F_SCALE',
       'T_SCALE', 'RATING_BASIS', 'WIND_SPEED', 'TEN_MIN_WIND_SPEED',
       'FUNNEL_SIGHTED', 'SUCTION_VORTICES', 'PRECIPITATION_AMOUNT',
       'SNOW_FALL_AMOUNT', 'PEAK_PRECIP_AMOUNT', 'PEAK_SNOW_FALL_AMOUNT',
       'PEAK_PRECIP_PERIOD', 'MAX_6_HOUR_PRECIP', 'MAX_6_HOUR_SNOW_FALL',
       'MAX_12_HOUR_PRECIP', 'MAX_12_HOUR_SNOW_FALL', 'MAX_24_HOUR_PRECIP',
       'MAX_24_HOUR_SNOW_FALL', 'CONVECTIVE', 'TOTAL_DURATION', 'TYPE_PRECIP',
       'SIZE_ACCOMPANYING_HAIL', 'POSSIBILITIES', 'PATH_LENGTH',
       'MEAN_PATH_WIDTH', 'MAX_PATH_WIDTH', 'MAX_VERTICAL_DEVELOP',
       'DIRECTION_MOVEMENT', 'SNOW_HAZARDS', 'MEAN_HEIGHT_SNOW_CORNICES',
       'MAX_HEIGHT_SNOW_CORNICES', 'ICE_HAZARDS', 'THICKNESS_ICE_COVER',
       'THICKNESS_RIME_COVER', 'AVALANCHE_TYPE', 'AVALANCHE_FLOW_TYPE',
       'SNOW_MASS_TYPE', 'AVALANCHE_SIZE', 'AVALANCHE_TRIGGER', 'PEAK_CURRENT']]

In [22]:
essl_modified = essl_modified.groupby(["TIME_EVENT", "TYPE_EVENT"], as_index=False).size()

In [23]:
essl_modified[essl_modified["TIME_EVENT"] == 1980]

,TIME_EVENT,TYPE_EVENT,size
0,1980,HAIL,6
1,1980,PRECIP,8
2,1980,SNOW,1
3,1980,TORNADO,27
4,1980,WIND,12


In [24]:
event_counts = essl_modified.reset_index().pivot_table(index='TIME_EVENT', columns='TYPE_EVENT', values="size", fill_value=0)
event_counts['Total'] = event_counts.sum(axis=1)

In [25]:
event_counts = event_counts.reset_index().rename_axis(None, axis=1)

In [26]:
event_counts.head(5)

,TIME_EVENT,DEVIL,GUSTNADO,HAIL,ICE,PRECIP,SNOW,TORNADO,WIND,Total
0,1980,0,0,6,0,8,1,27,12,54
1,1981,0,0,2,0,18,0,7,6,33
2,1982,0,0,4,0,1,0,9,17,31
3,1983,0,0,9,0,11,0,11,10,41
4,1984,2,0,17,0,11,0,14,26,70


In [27]:
event_counts.columns

Index(['TIME_EVENT', 'DEVIL', 'GUSTNADO', 'HAIL', 'ICE', 'PRECIP', 'SNOW',
       'TORNADO', 'WIND', 'Total'],
      dtype='object')

In [28]:
event_counts = event_counts.rename(columns={'TIME_EVENT':"Year", 'DEVIL':"Devil", 
                                            'GUSTNADO':"Gustnado", 'HAIL':"Hail", 
                                            'ICE':"Ice", 'PRECIP':"Precip", 'SNOW':"Snow",
                                            'TORNADO':"Tornado", 'WIND':'Wind'})

### Merging all datasets (FAO, DWD, ESSL, ESWD, RANDOM)

In [29]:
final_data = final_fao.merge(final_dwd, how="inner", left_on="Year", right_on="Year")
final_data = final_data.merge(co2, how="inner", left_on="Year", right_on="Year")
final_data = final_data.merge(event_counts, how="inner", left_on="Year", right_on="Year")

In [30]:
final_data = final_data.sort_values(by=["Year", "Item"], ignore_index=True)
final_data

,Year,Item,Yield (tonnes/ha),Change (C),Annual Mean Precipitation,Air Temp Mean,Frost Days,Hot Days,Ice Days,Summer Days,...,Mean CO2 (ppm),Devil,Gustnado,Hail,Ice,Precip,Snow,Tornado,Wind,Total
0,1980,Asparagus,2.2023,-0.190,771.58,7.63,103.61,1.45,30.73,16.18,...,338.762500,0,0,6,0,8,1,27,12,54
1,1980,Barley,4.3104,-0.190,771.58,7.63,103.61,1.45,30.73,16.18,...,338.762500,0,0,6,0,8,1,27,12,54
2,1980,"Beans, dry",2.4292,-0.190,771.58,7.63,103.61,1.45,30.73,16.18,...,338.762500,0,0,6,0,8,1,27,12,54
3,1980,"Broad beans and horse beans, dry",2.4565,-0.190,771.58,7.63,103.61,1.45,30.73,16.18,...,338.762500,0,0,6,0,8,1,27,12,54
4,1980,"Broad beans and horse beans, green",10.4409,-0.190,771.58,7.63,103.61,1.45,30.73,16.18,...,338.762500,0,0,6,0,8,1,27,12,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,2021,Sugar beet,81.7645,1.304,772.58,9.16,89.36,4.53,15.70,37.06,...,416.450833,2,0,464,13,1535,186,40,3762,6002
2041,2021,Sunflower seed,2.6031,1.304,772.58,9.16,89.36,4.53,15.70,37.06,...,416.450833,2,0,464,13,1535,186,40,3762,6002
2042,2021,Tomatoes,254.4250,1.304,772.58,9.16,89.36,4.53,15.70,37.06,...,416.450833,2,0,464,13,1535,186,40,3762,6002
2043,2021,Triticale,5.8136,1.304,772.58,9.16,89.36,4.53,15.70,37.06,...,416.450833,2,0,464,13,1535,186,40,3762,6002


In [31]:
final_data["Item"].unique()

array(['Asparagus', 'Barley', 'Beans, dry',
       'Broad beans and horse beans, dry',
       'Broad beans and horse beans, green', 'Cabbages',
       'Carrots and turnips', 'Cauliflowers and broccoli',
       'Cucumbers and gherkins', 'Gooseberries', 'Grapes', 'Hop cones',
       'Lentils, dry', 'Lettuce and chicory', 'Lupins', 'Maize (corn)',
       'Mixed grain', 'Mustard seed', 'Oats',
       'Onions and shallots, dry (excluding dehydrated)',
       'Other beans, green', 'Other oil seeds, n.e.c.',
       'Other pulses n.e.c.', 'Other vegetables, fresh n.e.c.',
       'Peas, dry', 'Peas, green', 'Poppy seed', 'Potatoes',
       'Rape or colza seed', 'Rye', 'Spinach', 'Sugar beet', 'Tomatoes',
       'Unmanufactured tobacco', 'Vetches', 'Wheat', 'Apples', 'Apricots',
       'Cherries', 'Currants', 'Peaches and nectarines', 'Pears',
       'Plums and sloes', 'Raspberries', 'Sour cherries', 'Strawberries',
       'Triticale', 'Walnuts, in shell',
       'Leeks and other alliaceous vege

In [32]:
masterList= {"Fruits and Nuts":['Apples', 'Apricots', 'Cherries', 'Currants', 'Gooseberries', 'Grapes', 'Peaches and nectarines', 'Pears', 'Plums and sloes', 'Raspberries', 'Sour cherries', 'Strawberries', 
                                       'Walnuts, in shell', 'Blueberries', 'Other stone fruits', 'Other berries and fruits of the genus vaccinium n.e.c.'],
            "Vegetables and Legumes":['Asparagus', 'Broad beans and horse beans, dry', 'Broad beans and horse beans, green', 'Beans, dry', 'Cabbages', 'Carrots and turnips', 'Cauliflowers and broccoli', 
                                                    'Cucumbers and gherkins', 'Lentils, dry', 'Lettuce and chicory', 'Lupins', 'Other beans, green', 'Other pulses n.e.c.', 'Other vegetables, fresh n.e.c.', 'Peas, dry', 
                                                    'Peas, green', 'Potatoes', 'Spinach', 'Sugar beet', 'Tomatoes', 'Leeks and other alliaceous vegetables', 'Chillies and peppers, green (Capsicum spp. and Pimenta spp.)', 
                                                    'Pumpkins, squash and gourds', 'Onions and shallots, green', 'Onions and shallots, dry (excluding dehydrated)'],
             "Cereals":['Barley', 'Maize (corn)', 'Mixed grain', 'Oats', 'Rye', 'Triticale', 'Wheat'],
             "Oilseed":['Mustard seed', 'Other oil seeds, n.e.c.', 'Poppy seed', 'Rape or colza seed', 'Sunflower seed', 'Soya beans', 'Linseed', ],
             "Others":['Hop cones','Vetches', 'Unmanufactured tobacco']}

In [33]:
# Define a lambda function to apply the condition
category_func = lambda x: next((k for k, v in masterList.items() if x in v), None)

# Apply the lambda function to create new column
final_data['Category'] = final_data['Item'].apply(category_func)

In [34]:
final_data["Category"].unique()

array(['Vegetables and Legumes', 'Cereals', 'Fruits and Nuts', 'Others',
       'Oilseed'], dtype=object)

In [35]:
print(final_data[final_data["Category"] == "Oilseed"].shape)
print(final_data[final_data["Category"] == "Cereals"].shape)
print(final_data[final_data["Category"] == "Vegetables and Legumes"].shape)
print(final_data[final_data["Category"] == "Others"].shape)
print(final_data[final_data["Category"] == "Fruits and Nuts"].shape)

(220, 23)
(289, 23)
(905, 23)
(95, 23)
(536, 23)


## Saving the final dataframe into a csv

In [36]:
final_data.sample(10)

,Year,Item,Yield (tonnes/ha),Change (C),Annual Mean Precipitation,Air Temp Mean,Frost Days,Hot Days,Ice Days,Summer Days,...,Devil,Gustnado,Hail,Ice,Precip,Snow,Tornado,Wind,Total,Category
1517,2011,Mixed grain,4.1790,1.108,677.60,9.64,83.31,4.27,14.49,38.37,...,6,1,243,0,656,0,34,973,1913,Cereals
169,1984,"Peas, green",6.0646,-0.115,743.99,7.97,95.85,3.30,17.03,17.97,...,2,0,17,0,11,0,14,26,70,Vegetables and Legumes
463,1990,Rape or colza seed,2.8907,1.565,734.15,9.49,61.30,5.91,11.46,31.03,...,0,0,2,0,4,0,9,58,73,Oilseed
1865,2017,Strawberries,9.5566,1.636,814.74,9.58,76.17,6.80,16.63,39.01,...,3,0,174,1,508,138,30,3176,4030,Fruits and Nuts
1568,2012,Linseed,1.4286,1.338,726.41,9.09,78.82,7.62,24.52,36.80,...,2,0,148,213,250,160,35,732,1540,Oilseed
1803,2016,Potatoes,44.4027,1.971,713.24,9.55,82.78,9.21,11.98,45.41,...,3,0,225,5,791,146,63,1547,2780,Vegetables and Legumes
24,1980,"Peas, dry",1.8692,-0.190,771.58,7.63,103.61,1.45,30.73,16.18,...,0,0,6,0,8,1,27,12,54,Vegetables and Legumes
1666,2014,Cauliflowers and broccoli,24.2978,2.506,713.49,10.33,56.81,6.14,9.86,34.15,...,0,0,193,22,686,12,50,702,1665,Vegetables and Legumes
1518,2011,Mustard seed,1.3983,1.108,677.60,9.64,83.31,4.27,14.49,38.37,...,6,1,243,0,656,0,34,973,1913,Oilseed
854,1998,Peaches and nectarines,3.5000,1.146,808.89,9.06,74.83,7.08,20.68,27.12,...,0,0,70,0,12,0,36,24,142,Fruits and Nuts


In [37]:
final_data.to_csv("./working_data/dataset.csv")